# Data Collection and Cleaning

Data has been collected from both the EPA for Air Quality Index data and from the CDC for birth data relating to weight.  

## Imports

In [1]:
import pandas as pd
import numpy as np

from pathlib import Path

## EPA Air Quality System (AQS) Data

Data is in individual CSV files that were generated by the `AQS_query.py` script in the project repo and served by the Air Quality System (AQS) API.  This script requires an email and API key from https://aqs.epa.gov/aqsweb/documents/data_api.htmlhttps://aqs.epa.gov/aqsweb/documents/data_api.html  Due to the large amount of data that needed to be collected, this task was split between our project team members to collect various years of data.  Each state for each set of years generates a CSV file that need to be read in and concatenated so we can use for our project.

In [2]:
data_path = Path('data/aqs_county_files/')

# create a list of all the AQI csv files
aqs_dfs = [
    pd.read_csv(file) for file in data_path.glob('AQS_county*') if file.is_file()
]

# concatenate the dataframes
aqs = pd.concat(aqs_dfs)

### Important note: use validity indicator to drop unusable rows

After spending time with the data dictionary and other various resources from AirData on the aspects of the data we were collecting we decided to filter out certain values.  So here we only want data that has been indicated as valid.

In [3]:
aqs.shape

(333008, 57)

In [4]:
aqs = aqs[aqs['validity_indicator']=='Y']

In [5]:
aqs.shape

(273633, 57)

### Borrow EPA standard and only take measurements where `valid_day_count` > 75% of days in year

The EPA generally uses 75% as the cutoff for aggregation measures, so we will follow that here.
* [Source](https://aqs.epa.gov/aqsweb/documents/about_aqs_data.html#data-aggregation-summarization)

In [6]:
.75*365

273.75

In [7]:
# aqs = aqs[aqs['valid_day_count'] > .75*365]

In [8]:
aqs.shape

(273633, 57)

### Columns to Drop

As we went through the various stages of looking at all the data that was coming from the API collection, and considering what parameters and data we were focusing on for this project, we collected a list of the columns from the data to drop:

In [9]:
drop = ['state_code',
 'county_code',
 'site_number',
 'parameter_code',
 'poc',
 'latitude',
 'longitude',
 'city', 'cbsa_code', 'cbsa', 'date_of_last_change',
 'first_max_value',
  'first_max_datetime',
  'second_max_value',
  'second_max_datetime',
  'third_max_value',
  'third_max_datetime',
  'fourth_max_value',
  'fourth_max_datetime',
  'first_max_nonoverlap_value',
  'first_max_n_o_datetime',
  'second_max_nonoverlap_value',
  'second_max_n_o_datetime',
  'ninety_ninth_percentile',
  'ninety_eighth_percentile',
  'ninety_fifth_percentile',
  'ninetieth_percentile',
  'seventy_fifth_percentile',
  'fiftieth_percentile',
  'tenth_percentile',
  'local_site_name',
  'site_address',
 'datum',
 'units_of_measure',
 'sample_duration',
 'secondary_exceedance_count',
 'pollutant_standard',
 'metric_used',
 'method',
 'event_type',
 'observation_count',
 'observation_percent',
 'required_day_count',
 'exceptional_data_count',
 'null_observation_count']

In [10]:
aqs.drop(columns=drop, inplace=True)

In [11]:
aqs.head()

,parameter,sample_duration_code,year,validity_indicator,valid_day_count,primary_exceedance_count,certification_indicator,arithmetic_mean,standard_deviation,state,county,Unnamed: 0
0,Carbon monoxide,1,2006,Y,357,0.0,Certified,0.606517,0.325145,Indiana,Allen,NaN
1,Carbon monoxide,Z,2006,Y,357,0.0,Certified,0.610952,0.299519,Indiana,Allen,NaN
2,Ozone,1,2006,Y,180,0.0,Certified,0.051078,0.012712,Indiana,Allen,NaN
3,Ozone,1,2006,Y,174,0.0,Certified,0.052517,0.012942,Indiana,Allen,NaN
4,Ozone,W,2006,Y,180,0.0,Certified,0.045844,0.012638,Indiana,Allen,NaN


In [12]:
aqs.columns

Index(['parameter', 'sample_duration_code', 'year', 'validity_indicator',
       'valid_day_count', 'primary_exceedance_count',
       'certification_indicator', 'arithmetic_mean', 'standard_deviation',
       'state', 'county', 'Unnamed: 0'],
      dtype='object')

### Check County and State Information

Mainly looking for outliers or any text in the names that might be cause for further investigation, especially since we will be merging this data with the CDC data based on year, state, and county.

In [13]:
# commented out to prevent long output in the notebook
aqs['county'].unique()

array(['Allen', 'Boone', 'Carroll', 'Clark', 'Daviess', 'Dearborn',
       'Delaware', 'Dubois', 'Elkhart', 'Floyd', 'Fountain', 'Gibson',
       'Greene', 'Hamilton', 'Hancock', 'Hendricks', 'Howard',
       'Huntington', 'Jackson', 'Jasper', 'Johnson', 'Knox', 'Lake',
       'LaPorte', 'Madison', 'Marion', 'Morgan', 'Perry', 'Pike',
       'Porter', 'Posey', 'St. Joseph', 'Shelby', 'Spencer', 'Tippecanoe',
       'Vanderburgh', 'Vigo', 'Wabash', 'Warrick', 'Wayne', 'Whitley',
       'Henry', 'Monroe', 'Adams', 'Allegheny', 'Armstrong', 'Beaver',
       'Berks', 'Blair', 'Bucks', 'Cambria', 'Centre', 'Chester',
       'Clearfield', 'Cumberland', 'Dauphin', 'Erie', 'Franklin',
       'Indiana', 'Lackawanna', 'Lancaster', 'Lawrence', 'Lebanon',
       'Lehigh', 'Luzerne', 'Lycoming', 'Mercer', 'Montgomery',
       'Northampton', 'Philadelphia', 'Somerset', 'Tioga', 'Warren',
       'Washington', 'Westmoreland', 'York', 'Elk', 'Bradford', 'Carbon',
       'Bexar', 'Bowie', 'Brazoria', 'B

In [14]:
aqs['state'].unique()

array(['Indiana', 'Pennsylvania', 'Texas', 'Mississippi', 'Wyoming',
       'Wisconsin', 'Nevada', 'Delaware', 'Missouri', 'Virgin Islands',
       'South Dakota', 'South Carolina', 'Vermont', 'Iowa', 'Illinois',
       'New Jersey', 'West Virginia', 'Maryland', 'California',
       'Puerto Rico', 'Kentucky', 'Rhode Island', 'Arkansas',
       'District Of Columbia', 'Oklahoma', 'Massachusetts', 'Ohio',
       'Idaho', 'Washington', 'Kansas', 'New Mexico', 'Alabama', 'Alaska',
       'Arizona', 'Colorado', 'Connecticut', 'Florida', 'Georgia',
       'Hawaii', 'Louisiana', 'Maine', 'Michigan', 'Minnesota', 'Montana',
       'Nebraska', 'New Hampshire', 'New York', 'North Carolina',
       'North Dakota', 'Oregon', 'Tennessee', 'Utah', 'Virginia'],
      dtype=object)

May want to drop locations such as:
* Puerto Rico

### Transform Data

The data came in as individual rows, but we need to transform/reshape the data so that the AQI pollutants are columns and the values match with the year/state/county they are reported in so that we can have a complete dataset for modeling.

EPA makes a recommendation to drop SO2 from data because it is more of a localized pollutant and can't be represented on a country or metropolitan area level.

In [15]:
# pivot = pd.pivot_table(aqs, values="arithmetic_mean", index=["state", "county", "year"], columns=["parameter"])
# 11707 rows × 8 columns

In [16]:
aqs = pd.pivot_table(aqs, values="arithmetic_mean", 
                       index=["state", "county", "year"], 
                       columns=["parameter"],
                       aggfunc='max')
aqs.shape

(15430, 8)

In [17]:
# drop columns with low counts of pollutants or identified by EPA as not
# representative

aqs.drop(columns=['Lead (TSP) LC', 'Lead PM10 LC FRM/FEM', 'Sulfur dioxide'], inplace=True)

In [18]:
# drop rows that don't have all pollutant data and reset index for merging later
aqs.dropna(inplace=True)
aqs.reset_index(inplace=True)

In [19]:
aqs

parameter,state,county,year,Carbon monoxide,Nitrogen dioxide (NO2),Ozone,PM10 Total 0-10um STP,PM2.5 - Local Conditions
0,Alabama,Jefferson,2014,0.578752,28.600838,0.048302,24.918033,11.698347
1,Alabama,Jefferson,2015,0.526681,22.163818,0.047164,23.883333,11.772881
2,Alabama,Jefferson,2016,0.444555,28.544643,0.051325,24.256809,10.781667
3,Alabama,Jefferson,2017,0.367024,20.712610,0.045971,22.300057,10.325455
4,Alabama,Jefferson,2018,0.344179,23.718644,0.049131,22.985533,10.329508
...,...,...,...,...,...,...,...,...
1437,Wyoming,Laramie,2020,0.135107,3.292139,0.042759,16.646483,5.070175
1438,Wyoming,Laramie,2021,0.085618,10.518927,0.051487,14.539488,5.286993
1439,Wyoming,Sweetwater,2011,0.408231,14.536723,0.054370,32.694915,5.110619
1440,Wyoming,Sweetwater,2012,0.364196,12.745042,0.055888,25.573770,6.997541


In [20]:
# pivot

In [21]:
# pivot = pd.pivot_table(aqs, values="arithmetic_mean", 
#                        index=["state", "county", "year"], 
#                        columns=["parameter"],
#                        aggfunc='max')
# pivot.shape

In [22]:
# pivot.drop(columns=['Lead (TSP) LC', 'Lead PM10 LC FRM/FEM', 'Sulfur dioxide'], inplace=True)

In [23]:
# pivot.dropna(inplace=True) # this model for Winston

In [24]:
# pivot.reset_index(inplace=True)

In [25]:
# pivot

In [26]:
# aqs['parameter'].value_counts()

In [27]:
aqs.shape

(1442, 8)

In [28]:
aqs['county'].unique()

array(['Jefferson', 'Fairbanks North Star ', 'Maricopa', 'Pima',
       'Pulaski', 'Alameda', 'Butte', 'Contra Costa', 'Fresno',
       'Humboldt', 'Imperial', 'Kern', 'Los Angeles', 'Madera', 'Marin',
       'Mendocino', 'Monterey', 'Napa', 'Orange', 'Riverside',
       'Sacramento', 'San Bernardino', 'San Diego', 'San Francisco',
       'San Joaquin', 'San Luis Obispo', 'San Mateo', 'Santa Barbara',
       'Santa Clara', 'Santa Cruz', 'Solano', 'Sonoma', 'Stanislaus',
       'Sutter', 'Yolo', 'Adams', 'Denver', 'La Plata', 'Fairfield',
       'Hartford', 'Litchfield', 'New Haven', 'New Castle',
       'District of Columbia', 'Broward', 'Duval', 'Hillsborough',
       'Miami-Dade', 'Palm Beach', 'Pinellas', 'DeKalb', 'Fulton',
       'Honolulu', 'Ada', 'Cook', 'Saint Clair', 'Lake', 'Marion',
       'Vanderburgh', 'Polk', 'Scott', 'Sedgwick', 'Wyandotte',
       'East Baton Rouge', 'Aroostook', 'Cumberland', 'Baltimore (City)',
       "Prince George's", 'Hampden', 'Suffolk', 'Worceste

## EPA Annual Air Quality Index (AQI) Summary Data

### Import Data

The air quality data was downloaded from https://aqs.epa.gov/aqsweb/airdata/download_files.html#Annual as files containing annual data from years 2006-2021.  We are combining these CSV files into a single dataframe for use in this project.  This is summary data that may also be useful in modeling and will be used in parallel with the AQS data above that was collected via API.

In [29]:
# code from Winston merged with approach from
# https://towardsdatascience.com/pandas-concat-tricks-you-should-know-to-speed-up-your-data-analysis-cd3d4fdfe6dd

data_path = Path('data/annual_summary_files/')

# create a list of all the AQI csv files
aqi_dfs = (
    pd.read_csv(file) for file in data_path.glob('annual_aqi_*') if file.is_file()
)

# concatenate the dataframes
aqi = pd.concat(aqi_dfs)

# export the final csv -- I think this was causing some duplication issues
# aqi_df.to_csv('data/annual_summary/annual_aqi_by_county_all.csv', index=False)

Read the final CSV for all the AQI data

In [30]:
# aqi = pd.read_csv('data/annual_summary/annual_aqi_by_county_all.csv')

### General Look and Cleanup

In [31]:
aqi.head()

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10
0,Alabama,Baldwin,2009,252,218,32,2,0,0,0,136,53,36,0,0,200,52,0
1,Alabama,Clay,2009,119,97,22,0,0,0,0,94,59,33,0,0,0,119,0
2,Alabama,Colbert,2009,323,220,103,0,0,0,0,76,60,43,0,0,132,191,0
3,Alabama,DeKalb,2009,363,311,52,0,0,0,0,100,54,36,0,0,308,55,0
4,Alabama,Elmore,2009,244,228,16,0,0,0,0,80,49,36,0,0,244,0,0


In [32]:
aqi.columns

Index(['State', 'County', 'Year', 'Days with AQI', 'Good Days',
       'Moderate Days', 'Unhealthy for Sensitive Groups Days',
       'Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days', 'Max AQI',
       '90th Percentile AQI', 'Median AQI', 'Days CO', 'Days NO2',
       'Days Ozone', 'Days PM2.5', 'Days PM10'],
      dtype='object')

In [33]:
aqi.shape

(16751, 18)

In [34]:
aqi.dtypes

State                                  object
County                                 object
Year                                    int64
Days with AQI                           int64
Good Days                               int64
Moderate Days                           int64
Unhealthy for Sensitive Groups Days     int64
Unhealthy Days                          int64
Very Unhealthy Days                     int64
Hazardous Days                          int64
Max AQI                                 int64
90th Percentile AQI                     int64
Median AQI                              int64
Days CO                                 int64
Days NO2                                int64
Days Ozone                              int64
Days PM2.5                              int64
Days PM10                               int64
dtype: object

In [35]:
aqi.columns = [col.lower().replace(' ', '_') for col in aqi.columns]

In [36]:
aqi.columns

Index(['state', 'county', 'year', 'days_with_aqi', 'good_days',
       'moderate_days', 'unhealthy_for_sensitive_groups_days',
       'unhealthy_days', 'very_unhealthy_days', 'hazardous_days', 'max_aqi',
       '90th_percentile_aqi', 'median_aqi', 'days_co', 'days_no2',
       'days_ozone', 'days_pm2.5', 'days_pm10'],
      dtype='object')

### Rename Columns
Mainly to trim the length of some of the column names to make it easier to work with.

In [37]:
aqi.rename(columns={
    'moderate_days': 'mod_days',
    'unhealthy_for_sensitive_groups_days': 'uh_sens_days',
    'unhealthy_days': 'uh_days',
    'very_unhealthy_days': 'v_uh_days',
    'hazardous_days': 'haz_days',
    '90th_percentile_aqi': '90_per_aqi'
    }, inplace=True)

In [38]:
aqi.isnull().sum()[aqi.isnull().sum() > 0]

Series([], dtype: int64)

### Check County and State Information

Mainly looking for outliers or any text in the names that might be cause for further investigation, especially since we will be merging this data with the CDC data based on year, state, and county.

In [39]:
aqi[['state', 'county']].nunique()

state      55
county    913
dtype: int64

In [40]:
aqi['state'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Country Of Mexico', 'Delaware',
       'District Of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virgin Islands', 'Virginia', 'Washington', 'West Virginia',
       'Wisconsin', 'Wyoming', 'Canada'], dtype=object)

May want to drop locations such as:
* Country of Mexico
* Puerto Rico
* Virgin Islands
* Canada

In [41]:
aqi.drop(
    aqi[
        aqi['state'].isin(
            ['Country Of Mexico', 'Puerto Rico', 'Virgin Islands', 'Canada']
        )
    ].index, inplace=True)

In [42]:
# commented out to save space in the published notebook
# aqi['county'].unique()

In [43]:
aqi.head(3)

,state,county,year,days_with_aqi,good_days,mod_days,uh_sens_days,uh_days,v_uh_days,haz_days,max_aqi,90_per_aqi,median_aqi,days_co,days_no2,days_ozone,days_pm2.5,days_pm10
0,Alabama,Baldwin,2009,252,218,32,2,0,0,0,136,53,36,0,0,200,52,0
1,Alabama,Clay,2009,119,97,22,0,0,0,0,94,59,33,0,0,0,119,0
2,Alabama,Colbert,2009,323,220,103,0,0,0,0,76,60,43,0,0,132,191,0


In [44]:
aqi.describe()

,year,days_with_aqi,good_days,mod_days,uh_sens_days,uh_days,v_uh_days,haz_days,max_aqi,90_per_aqi,median_aqi,days_co,days_no2,days_ozone,days_pm2.5,days_pm10
count,14831.000000,14831.000000,14831.000000,14831.000000,14831.000000,14831.000000,14831.000000,14831.000000,14831.000000,14831.000000,14831.000000,14831.000000,14831.000000,14831.000000,14831.000000,14831.000000
mean,2013.363158,304.668397,233.027443,65.437192,5.110916,0.942957,0.099386,0.050502,128.249343,61.747691,37.717888,0.990021,6.169442,171.109298,112.097431,14.302205
std,4.607362,90.612711,83.426559,51.980877,10.695543,4.180807,1.156806,0.710635,234.704761,19.622489,11.066944,12.070323,25.545751,119.476797,109.382057,53.553639
min,2006.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2009.000000,245.000000,175.000000,25.000000,0.000000,0.000000,0.000000,0.000000,90.000000,50.000000,34.000000,0.000000,0.000000,51.000000,0.000000,0.000000
50%,2013.000000,360.000000,247.000000,53.000000,1.000000,0.000000,0.000000,0.000000,112.000000,59.000000,39.000000,0.000000,0.000000,188.000000,92.000000,0.000000
75%,2017.000000,365.000000,303.000000,94.000000,6.000000,0.000000,0.000000,0.000000,145.000000,71.000000,44.000000,0.000000,0.000000,245.000000,181.000000,1.000000
max,2021.000000,366.000000,365.000000,339.000000,122.000000,92.000000,74.000000,37.000000,14043.000000,306.000000,132.000000,365.000000,365.000000,366.000000,366.000000,366.000000


All the values seem like they would be in range--meaning no negative counts, no days over 366.

## CDC Data

Want to import the CDC data and compare the counties.

We are using the WONDER tool from the CDC to export the files that we are importing to this notebook. For both files, we filtered out any births with maternal risk factors so we they are not influencing birth weight.  There are some years that have no or suppressed data, which are omitted from the CDC export and are not present.We filtered out any births with maternal risk factors so we they are not influencing birth weight.  There are some years that have no or suppressed data, which are omitted from the CDC export and are not present. 

There are two main files:
1. `Natality_low_2007-2021` which represents all the births below 2500 grams by year and county. 
1. `Natality_all_2007-2021` which represents all the births by year and county.

### Import Data

In [45]:
cdc_low = pd.read_csv('data/cdc_files/Natality_low_2007-2021.txt', sep='\t')
cdc_all = pd.read_csv('data/cdc_files/Natality_all_2007-2021.txt', sep='\t')

### General Look and Cleanup

In [46]:
cdc_low.head()

,Notes,State,State Code,County,County Code,Year,Year Code,Births,Average Birth Weight,Average LMP Gestational Age,Average OE Gestational Age
0,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2014.0,2014.0,137.0,1995.255,34.241,34.051
1,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2015.0,2015.0,141.0,1832.837,32.965,32.922
2,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2016.0,2016.0,120.0,2008.358,34.467,34.058
3,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2017.0,2017.0,100.0,2082.830,34.550,34.380
4,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2018.0,2018.0,122.0,1942.303,33.762,33.508


In [47]:
cdc_all.head()

,Notes,State,State Code,County,County Code,Year,Year Code,Births,Average Birth Weight,Average LMP Gestational Age,Average OE Gestational Age
0,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2014.0,2014.0,1724.0,3296.472,38.595,38.489
1,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2015.0,2015.0,1857.0,3290.811,38.487,38.347
2,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2016.0,2016.0,1722.0,3315.387,38.690,38.497
3,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2017.0,2017.0,1782.0,3346.588,38.713,38.572
4,NaN,Alabama,1.0,"Baldwin County, AL",1003.0,2018.0,2018.0,1770.0,3301.324,38.580,38.423


In [48]:
cdc_low.dtypes

Notes                           object
State                           object
State Code                     float64
County                          object
County Code                    float64
Year                           float64
Year Code                      float64
Births                         float64
Average Birth Weight           float64
Average LMP Gestational Age    float64
Average OE Gestational Age     float64
dtype: object

In [49]:
cdc_all.dtypes

Notes                           object
State                           object
State Code                     float64
County                          object
County Code                    float64
Year                           float64
Year Code                      float64
Births                         float64
Average Birth Weight           float64
Average LMP Gestational Age    float64
Average OE Gestational Age     float64
dtype: object

In [50]:
cdc_low.shape

(6712, 11)

In [51]:
cdc_all.shape

(6715, 11)

In [52]:
cdc_low.columns, cdc_all.columns

(Index(['Notes', 'State', 'State Code', 'County', 'County Code', 'Year',
        'Year Code', 'Births', 'Average Birth Weight',
        'Average LMP Gestational Age', 'Average OE Gestational Age'],
       dtype='object'),
 Index(['Notes', 'State', 'State Code', 'County', 'County Code', 'Year',
        'Year Code', 'Births', 'Average Birth Weight',
        'Average LMP Gestational Age', 'Average OE Gestational Age'],
       dtype='object'))

In [53]:
cdc_low.columns = [col.lower().replace(' ', '_') for col in cdc_low.columns]
cdc_all.columns = [col.lower().replace(' ', '_') for col in cdc_all.columns]

Renaming the % of births column, and if we want different names for other columns, we can do it at this step

In [54]:
cdc_low.columns

Index(['notes', 'state', 'state_code', 'county', 'county_code', 'year',
       'year_code', 'births', 'average_birth_weight',
       'average_lmp_gestational_age', 'average_oe_gestational_age'],
      dtype='object')

In [55]:
cdc_low['notes'].nunique(), cdc_all['notes'].nunique()

(91, 87)

In [56]:
# cdc_all['notes'].unique()

All the notes look like they are what shows up at the bottom of the file and isn't data that we're looking for.  Dropping the notes column

In [57]:
cdc_low.drop(['notes'], axis=1, inplace=True)
cdc_all.drop(['notes'], axis=1, inplace=True)

In [58]:
cdc_low.isna().sum()[cdc_low.isna().sum() > 0]

state                          96
state_code                     96
county                         96
county_code                    96
year                           96
year_code                      96
births                         96
average_birth_weight           96
average_lmp_gestational_age    96
average_oe_gestational_age     96
dtype: int64

In [59]:
cdc_all.isna().sum()[cdc_all.isna().sum() > 0]

state                          92
state_code                     92
county                         92
county_code                    92
year                           92
year_code                      92
births                         92
average_birth_weight           92
average_lmp_gestational_age    92
average_oe_gestational_age     92
dtype: int64

In [60]:
cdc_low.tail(3)

,state,state_code,county,county_code,year,year_code,births,average_birth_weight,average_lmp_gestational_age,average_oe_gestational_age
6709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
cdc_all.tail(3)

,state,state_code,county,county_code,year,year_code,births,average_birth_weight,average_lmp_gestational_age,average_oe_gestational_age
6712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We need state and county information for this project,and it looks like all these null values belong, possibly to rows that were holding that notes data.  Dropping these rows to see if that clears up all the nulls.

Drop the rows that have the footnotes in them

In [62]:
cdc_low.dropna(subset=['state'], inplace=True)
cdc_all.dropna(subset=['state'], inplace=True)

In [63]:
cdc_low.isna().sum()[cdc_low.isna().sum() > 0]

Series([], dtype: int64)

In [64]:
cdc_all.isna().sum()[cdc_all.isna().sum() > 0]

Series([], dtype: int64)

That did resolve all the null values.

In [65]:
cdc_low.shape, cdc_all.shape

((6616, 10), (6623, 10))

In [66]:
cdc_low.tail(3)

,state,state_code,county,county_code,year,year_code,births,average_birth_weight,average_lmp_gestational_age,average_oe_gestational_age
6613,Wyoming,56.0,"Unidentified Counties, WY",56999.0,2019.0,2019.0,335.0,2062.824,35.051,34.919
6614,Wyoming,56.0,"Unidentified Counties, WY",56999.0,2020.0,2020.0,307.0,2088.586,35.365,34.987
6615,Wyoming,56.0,"Unidentified Counties, WY",56999.0,2021.0,2021.0,319.0,2063.197,34.821,34.862


In [67]:
cdc_all.tail(3)

,state,state_code,county,county_code,year,year_code,births,average_birth_weight,average_lmp_gestational_age,average_oe_gestational_age
6620,Wyoming,56.0,"Unidentified Counties, WY",56999.0,2019.0,2019.0,4592.0,3218.040,38.709,38.636
6621,Wyoming,56.0,"Unidentified Counties, WY",56999.0,2020.0,2020.0,4285.0,3228.561,38.733,38.641
6622,Wyoming,56.0,"Unidentified Counties, WY",56999.0,2021.0,2021.0,4379.0,3227.242,38.608,38.551


It looks like year and year_code might contain the same data

In [68]:
(cdc_low['year'] == cdc_low['year_code']).sum() == cdc_low.shape[0]

True

`year` and `year_code` appear to be the same.  We also don't have state or county codes in the EPA data, so dropping those as well.

In [69]:
cdc_low.drop(['year_code', 'county_code', 'state_code'], axis=1, inplace=True)
cdc_all.drop(['year_code', 'county_code', 'state_code'], axis=1, inplace=True)

Also want the year as an int not a float

In [70]:
cdc_low['year'] = cdc_low['year'].astype(int)
cdc_all['year'] = cdc_all['year'].astype(int)

In [71]:
cdc_low.columns

Index(['state', 'county', 'year', 'births', 'average_birth_weight',
       'average_lmp_gestational_age', 'average_oe_gestational_age'],
      dtype='object')

### Gestational Age at Birth
Beginning in 2014 NCHS changed the standard for gestational period from the Last Menstrual Period (LMP) based gestational age to the Obstetric/clinical Estimate (OE) based gestational age. Obstetric/clinical Estimate (OE) based gestational age groups are available for years 2007 and later in WONDER since February 2016. Refer to [Measuring Gestational Age in Vital Statistics Data: Transitioning to the Obstetric Estimate](http://www.cdc.gov/nchs/data/nvsr/nvsr64/nvsr64_05.pdf) for more information.

I believe this means this means we should drop the LMP column and keep the OE column.

In [72]:
cdc_low.drop(['average_lmp_gestational_age'], axis=1, inplace=True)
cdc_all.drop(['average_lmp_gestational_age'], axis=1, inplace=True)

In [73]:
cdc_low.columns

Index(['state', 'county', 'year', 'births', 'average_birth_weight',
       'average_oe_gestational_age'],
      dtype='object')

### Rename Columns

To prepare for merging this data, giving the columns specific names to indicate which dataframe they came from.

In [74]:
cdc_low.rename(columns={
    'births': 'births_low',
    'average_birth_weight': 'avg_weight_low',
    'average_oe_gestational_age': 'avg_ges_age_low'
    }, inplace=True)

cdc_all.rename(columns={
    'births': 'births_all',
    'average_birth_weight': 'avg_weight_all',
    'average_oe_gestational_age': 'avg_ges_age_all'
    }, inplace=True)

In [75]:
cdc_low.head(3)

,state,county,year,births_low,avg_weight_low,avg_ges_age_low
0,Alabama,"Baldwin County, AL",2014,137.0,1995.255,34.051
1,Alabama,"Baldwin County, AL",2015,141.0,1832.837,32.922
2,Alabama,"Baldwin County, AL",2016,120.0,2008.358,34.058


In [76]:
cdc_all.head(3)

,state,county,year,births_all,avg_weight_all,avg_ges_age_all
0,Alabama,"Baldwin County, AL",2014,1724.0,3296.472,38.489
1,Alabama,"Baldwin County, AL",2015,1857.0,3290.811,38.347
2,Alabama,"Baldwin County, AL",2016,1722.0,3315.387,38.497


### Merge CDC Data

Before further cleaning, the dataframes will be merged.  The low birthweight dataframe `cdc_low` is the main data and we will supplement that with the all birthweight data `cdc_all` so we should end up with a single dataframe that contains 1 entry for each year/county combination that has both the low birthweight and all birthweight data.

In [77]:
cdc = pd.merge(cdc_low, cdc_all, how='left', on=['state', 'county', 'year'])

In [78]:
cdc.head()

,state,county,year,births_low,avg_weight_low,avg_ges_age_low,births_all,avg_weight_all,avg_ges_age_all
0,Alabama,"Baldwin County, AL",2014,137.0,1995.255,34.051,1724.0,3296.472,38.489
1,Alabama,"Baldwin County, AL",2015,141.0,1832.837,32.922,1857.0,3290.811,38.347
2,Alabama,"Baldwin County, AL",2016,120.0,2008.358,34.058,1722.0,3315.387,38.497
3,Alabama,"Baldwin County, AL",2017,100.0,2082.830,34.380,1782.0,3346.588,38.572
4,Alabama,"Baldwin County, AL",2018,122.0,1942.303,33.508,1770.0,3301.324,38.423


### County Names

In [79]:
cdc['county'].sample(10)

2299        Linn County, IA
1888       Macon County, IL
1163     Collier County, FL
1890       Macon County, IL
3361      Greene County, MO
4767      Lorain County, OH
451       Merced County, CA
1501    Cherokee County, GA
4961        Wood County, OH
1827     Kendall County, IL
Name: county, dtype: object

I ran the sample code above multiple times to see some of the data in the county column.  Looks like they are including the state abbreviation in the County name, so will remove that.  There were also some places that didn't say "County" and said "Counties" (i.e. Unidentified Counties, WY) and also there were some locations that were parishes (in LA) or other unique monikers like Borough.  So I'm splitting on both the " Count" to capture County and Counties as well as "," to get both cases.

We should look at what the AQI data has and decide whether we need to treat the remaining unique naming cases that remain after this clean up.

In [80]:
cdc['county'] = cdc['county'
                   ].apply(lambda x: x.split(' Count')[0].split(',')[0].title())

In [81]:
cdc[['state', 'county']].nunique()

state      50
county    466
dtype: int64

In [82]:
cdc['county'].sample(10)

5389    Providence
2936     Middlesex
6121    Chittenden
410         Madera
2291       Johnson
1233          Lake
3039       Genesee
2043       Elkhart
3270        Desoto
3975       Niagara
Name: county, dtype: object

This looks more like what I would expect where it is just the name of the county.  Since we also saw that there were unidentified counties, and they are all just listed as Unidentified now, we should drop that data since we are trying to investigate county air quality and county birth weights, so without a specific county, the data is not valuable to us.

In [83]:
# looking through all the unique values for anything else interesting
# commented out to save space in the published notebook
# cdc['county'].unique()

I'm also seeing many counties that end in lower case 'city' so investigating some of those, like:
* St. Louis city
* Chesapeake city
* Norfolk city
* Portsmouth city

St. Louis is interesting, since it sounds like the city is a county, but there is also a St. Louis county that surrounds the city according to https://www.stlouis-mo.gov/government/about/city-government-structure.cfm

Chesapeake, Virginia is also an independent city and not part of a county https://en.wikipedia.org/wiki/Chesapeake,_Virginia

Given this information and the information above about seeing some "parish" listings in Louisiana, we checked against the AQI county listings and found that in the AQI data:
* the word Parish does not appear, but the name of the Parish does--like 'East Baton Rouge', so we should drop 'Parish' from the CDC county names.
* The word City does appear in the AQI data, but it is capitalized, so we should title case the CDC data or make all of them lower case when we go to match them up (this has been incorporated into earlier cleaning steps and is no longer a concern)

In [84]:
# drop parish
cdc['county'] = cdc['county'
                   ].apply(lambda x: x.split(' Parish')[0])

In [85]:
cdc.shape

(6616, 9)

In [86]:
cdc.drop(cdc[cdc['county'] == 'Unidentified'].index, inplace=True)

In [87]:
cdc.shape

(6094, 9)

In [88]:
cdc.dtypes

state               object
county              object
year                 int64
births_low         float64
avg_weight_low     float64
avg_ges_age_low    float64
births_all         float64
avg_weight_all     float64
avg_ges_age_all    float64
dtype: object

### Cleaning up county and state names before merging CDC and AQS/AQI data

#### By hand fixing saint vs. st. discrepancy

In [89]:
aqs['county'] = aqs['county'].str.replace('Saint Louis', 'St. Louis')
aqs['county'] = aqs['county'].str.replace('Saint Clair', 'St. Clair')

In [90]:
cdc_county = set(cdc['county'].unique())
aqs_county = set(aqs['county'].unique())
aqi_county = set(aqi['county'].unique())
len(cdc_county) - len(aqs_county)

313

In [91]:
cdc_county = set([county.lower().strip().replace('(','').replace(')','') for county in cdc_county])
aqs_county = set([county.lower().strip().replace('(','').replace(')','') for county in aqs_county])
aqi_county = set([county.lower().strip().replace('(','').replace(')','') for county in aqi_county])

In [92]:
aqs_county.difference(cdc_county)

{'adair',
 'aroostook',
 'burleigh',
 'charleston',
 'converse',
 'fairbanks north star',
 'fremont',
 'greenville',
 'la plata',
 'laramie',
 'mendocino',
 'rutland',
 'sutter',
 'sweetwater',
 'teton',
 'uintah'}

In [93]:
cdc_states = set(cdc['state'].unique())
aqs_states = set(aqs['state'].unique())
aqi_states = set(aqi['state'].unique())
len(cdc_states), len(aqs_states)

(49, 49)

In [94]:
cdc_states = set([state.lower().strip() for state in cdc_states])
aqs_states = set([state.lower().strip() for state in aqs_states])
aqi_states = set([state.lower().strip() for state in aqi_states])

In [95]:
cdc_states.difference(aqs_states)

{'nebraska', 'west virginia'}

In [96]:
cdc_state_dict = {State: state for State, state in zip(sorted(set(cdc['state'].unique())), sorted(cdc_states))}
cdc_county_dict = {County: county for County, county in zip(sorted(set(cdc['county'].unique())), sorted(cdc_county))}
aqs_state_dict = {State: state for State, state in zip(sorted(set(aqs['state'].unique())), sorted(aqs_states))}
aqs_county_dict = {County: county for County, county in zip(sorted(set(aqs['county'].unique())), sorted(aqs_county))}
aqi_state_dict = {State: state for State, state in zip(sorted(set(aqi['state'].unique())), sorted(aqi_states))}
aqi_county_dict = {County: county for County, county in zip(sorted(set(aqi['county'].unique())), sorted(aqi_county))}

In [97]:
cdc['state'] = cdc['state'].map(cdc_state_dict)
aqs['state'] = aqs['state'].map(aqs_state_dict)
cdc['county'] = cdc['county'].map(cdc_county_dict)
aqs['county'] = aqs['county'].map(aqs_county_dict)
aqi['county'] = aqi['county'].map(aqi_county_dict)
aqi['state'] = aqi['state'].map(aqi_state_dict)

In [98]:
cdc[['state','county']].isna().sum(), aqs[['state','county']].isna().sum(), aqi[['state','county']].isna().sum()

(state     0
 county    0
 dtype: int64,
 parameter
 state     0
 county    0
 dtype: int64,
 state     0
 county    0
 dtype: int64)

### Calculate Percent of Low Birthweight

We have the total births and the low birthweight births, so let's create a percentage of low birthrate births before exporting the cleaned data.

The current national average is 8.24% (.0824), however this includes all high-risk health groups. In order to adequately treat the group we are looking at we will calculate the average from our data and create a boolean feature as well to indicate if the observation is above the national average

In [99]:
cdc['pct_low'] = (cdc['births_low'] / cdc['births_all']).round(4)

In [163]:
cdc.groupby('year')[['births_low', 'births_all']].sum()

,births_low,births_all
year,,
2007,73395.0,1095143.0
2008,75358.0,1133511.0
2009,75181.0,1135931.0
2010,89907.0,1354409.0
2011,109603.0,1626495.0
2012,116248.0,1762440.0
2013,131708.0,1985625.0
2014,150585.0,2285703.0
2015,152414.0,2307588.0


In [100]:
nat_avg_rate = round(cdc[cdc['year']==2021]['births_low'].sum()/cdc[cdc['year']==2021]['births_all'].sum(), 4)
nat_avg_rate

0.0688

In [101]:
cdc['high_rate'] = cdc['pct_low'].apply(lambda x: 1 if x > nat_avg_rate else 0)

In [102]:
cdc.head(10)

,state,county,year,births_low,avg_weight_low,avg_ges_age_low,births_all,avg_weight_all,avg_ges_age_all,pct_low,high_rate
0,alabama,baldwin,2014,137.0,1995.255,34.051,1724.0,3296.472,38.489,0.0795,1
1,alabama,baldwin,2015,141.0,1832.837,32.922,1857.0,3290.811,38.347,0.0759,1
2,alabama,baldwin,2016,120.0,2008.358,34.058,1722.0,3315.387,38.497,0.0697,1
3,alabama,baldwin,2017,100.0,2082.830,34.380,1782.0,3346.588,38.572,0.0561,0
4,alabama,baldwin,2018,122.0,1942.303,33.508,1770.0,3301.324,38.423,0.0689,1
5,alabama,baldwin,2019,99.0,2060.768,34.919,1845.0,3327.293,38.521,0.0537,0
6,alabama,baldwin,2020,113.0,2025.478,34.982,1763.0,3305.012,38.529,0.0641,0
7,alabama,baldwin,2021,120.0,1969.508,34.269,1969.0,3293.984,38.394,0.0609,0
8,alabama,calhoun,2014,59.0,1972.661,34.475,948.0,3275.708,38.668,0.0622,0
9,alabama,calhoun,2015,65.0,1909.708,33.892,983.0,3267.121,38.590,0.0661,0


In [103]:
cdc['high_rate'].value_counts(normalize=True)

0    0.679357
1    0.320643
Name: high_rate, dtype: float64

## Merge CDC and EPA Data

Since we've found that the AQS data has more detail, but we have fewer counties that have a larger set of pollutant data, we will create both a set of merged data from the AQS API data and a set from the AQI Annual Summary data.

Both will be merged in a way where the current year of air quality data (AQS or AQI) will be merged with the following year CDC data.  The assumption here is that the impacts of poor air quality during pregnancy will appear in the following year's birth weight data and then any models that we build to classify or regress this data will have some predictive power for the following year based on the current year's air quality.

In [104]:
# county_aqs.head(3)

### Create year_for_merge column in the CDC data

In [105]:
cdc['year_for_merge'] = cdc['year'] -1

In [106]:
cdc.head(3)

,state,county,year,births_low,avg_weight_low,avg_ges_age_low,births_all,avg_weight_all,avg_ges_age_all,pct_low,high_rate,year_for_merge
0,alabama,baldwin,2014,137.0,1995.255,34.051,1724.0,3296.472,38.489,0.0795,1,2013
1,alabama,baldwin,2015,141.0,1832.837,32.922,1857.0,3290.811,38.347,0.0759,1,2014
2,alabama,baldwin,2016,120.0,2008.358,34.058,1722.0,3315.387,38.497,0.0697,1,2015


### Merge AQS and CDC Data

In [107]:
aqs_merged = pd.merge(aqs, cdc, how='inner', 
                  left_on=['state', 'county', 'year'],
                  right_on=['state', 'county', 'year_for_merge'])

In [108]:
aqs_merged.head(3)

,state,county,year_x,Carbon monoxide,Nitrogen dioxide (NO2),Ozone,PM10 Total 0-10um STP,PM2.5 - Local Conditions,year_y,births_low,avg_weight_low,avg_ges_age_low,births_all,avg_weight_all,avg_ges_age_all,pct_low,high_rate,year_for_merge
0,alabama,jefferson,2014,0.578752,28.600838,0.048302,24.918033,11.698347,2015,687.0,1940.866,34.068,7376.0,3216.859,38.508,0.0931,1,2014
1,alabama,jefferson,2015,0.526681,22.163818,0.047164,23.883333,11.772881,2016,681.0,1903.761,33.633,7032.0,3210.182,38.434,0.0968,1,2015
2,alabama,jefferson,2016,0.444555,28.544643,0.051325,24.256809,10.781667,2017,709.0,1947.176,34.059,7061.0,3188.066,38.456,0.1004,1,2016


In [109]:
aqs_merged.shape

(1012, 18)

In [110]:
aqs_merged.columns

Index(['state', 'county', 'year_x', 'Carbon monoxide',
       'Nitrogen dioxide (NO2)', 'Ozone', 'PM10 Total 0-10um STP',
       'PM2.5 - Local Conditions', 'year_y', 'births_low', 'avg_weight_low',
       'avg_ges_age_low', 'births_all', 'avg_weight_all', 'avg_ges_age_all',
       'pct_low', 'high_rate', 'year_for_merge'],
      dtype='object')

In [111]:
aqs_merged.drop(['year_y', 'year_for_merge'],
            axis=1, 
            inplace=True)

aqs_merged.rename(columns={'year_x': 'year'}, inplace=True)

In [112]:
aqs_merged.head(3)

,state,county,year,Carbon monoxide,Nitrogen dioxide (NO2),Ozone,PM10 Total 0-10um STP,PM2.5 - Local Conditions,births_low,avg_weight_low,avg_ges_age_low,births_all,avg_weight_all,avg_ges_age_all,pct_low,high_rate
0,alabama,jefferson,2014,0.578752,28.600838,0.048302,24.918033,11.698347,687.0,1940.866,34.068,7376.0,3216.859,38.508,0.0931,1
1,alabama,jefferson,2015,0.526681,22.163818,0.047164,23.883333,11.772881,681.0,1903.761,33.633,7032.0,3210.182,38.434,0.0968,1
2,alabama,jefferson,2016,0.444555,28.544643,0.051325,24.256809,10.781667,709.0,1947.176,34.059,7061.0,3188.066,38.456,0.1004,1


### Merge AQI and CDC Data

In [113]:
aqi_merged = pd.merge(aqi, cdc, how='inner', 
                  left_on=['state', 'county', 'year'],
                  right_on=['state', 'county', 'year_for_merge'])

In [114]:
aqi_merged.head(3)

,state,county,year_x,days_with_aqi,good_days,mod_days,uh_sens_days,uh_days,v_uh_days,haz_days,...,year_y,births_low,avg_weight_low,avg_ges_age_low,births_all,avg_weight_all,avg_ges_age_all,pct_low,high_rate,year_for_merge
0,california,alameda,2009,365,190,162,12,1,0,0,...,2010,1136.0,1980.992,34.260,17700.0,3314.900,38.817,0.0642,0,2009
1,california,butte,2009,365,185,148,31,1,0,0,...,2010,90.0,1963.467,33.809,1907.0,3390.433,38.850,0.0472,0,2009
2,california,contra costa,2009,365,274,84,6,1,0,0,...,2010,669.0,2002.323,34.283,10976.0,3330.618,38.823,0.0610,0,2009


In [115]:
aqi_merged.shape

(4635, 28)

In [116]:
aqi_merged.columns

Index(['state', 'county', 'year_x', 'days_with_aqi', 'good_days', 'mod_days',
       'uh_sens_days', 'uh_days', 'v_uh_days', 'haz_days', 'max_aqi',
       '90_per_aqi', 'median_aqi', 'days_co', 'days_no2', 'days_ozone',
       'days_pm2.5', 'days_pm10', 'year_y', 'births_low', 'avg_weight_low',
       'avg_ges_age_low', 'births_all', 'avg_weight_all', 'avg_ges_age_all',
       'pct_low', 'high_rate', 'year_for_merge'],
      dtype='object')

In [117]:
aqi_merged.drop(['year_y', 'year_for_merge'],
            axis=1, 
            inplace=True)

aqi_merged.rename(columns={'year_x': 'year'}, inplace=True)

In [118]:
aqi_merged.head(3)

,state,county,year,days_with_aqi,good_days,mod_days,uh_sens_days,uh_days,v_uh_days,haz_days,...,days_pm2.5,days_pm10,births_low,avg_weight_low,avg_ges_age_low,births_all,avg_weight_all,avg_ges_age_all,pct_low,high_rate
0,california,alameda,2009,365,190,162,12,1,0,0,...,234,0,1136.0,1980.992,34.260,17700.0,3314.900,38.817,0.0642,0
1,california,butte,2009,365,185,148,31,1,0,0,...,101,0,90.0,1963.467,33.809,1907.0,3390.433,38.850,0.0472,0
2,california,contra costa,2009,365,274,84,6,1,0,0,...,78,0,669.0,2002.323,34.283,10976.0,3330.618,38.823,0.0610,0


## Export Cleaned Data

In [119]:
# cdc.to_csv('data/cdc_cleaned.csv', index=False)
aqs_merged.to_csv('data/aqs_by_county_clean.csv', index=False)
aqi_merged.to_csv('data/annual_aqi_clean.csv', index=False)

# PARKING LOT

In [119]:
# merged[['state', 'county']].nunique()

In [120]:
# merged['high_rate'].value_counts(normalize=True)

In [121]:
# merged['state'].value_counts()

In [122]:
# pivot.drop(columns=['PM10 Total 0-10um STP']).dropna()

In [123]:
# all_params = pivot.dropna().reset_index()

In [124]:
# all_params

Pivot table
Get index from pivot table
re-index original 
get primary_exceedance_count from this

In [125]:
# pivot.columns

In [126]:
# pivot[['PM2.5 - Local Conditions', 'Ozone']].dropna()

Unstack approach

In [127]:
# county_aqs = aqs.groupby(['state', 'county', 'year', 'parameter'])['arithmetic_mean'].mean().unstack().reset_index()

In [128]:
# county_aqs.head(20)

In [129]:
# county_aqs[['Ozone', 'Sulfur dioxide']].dropna()

Looking at example data that Winston collected via API

In [130]:
# df.drop_duplicates()

In [131]:
# df = pd.read_csv('data/AQS_data_test_Alabama_2006.csv')

In [132]:
# df.head()

In [133]:
# df.columns

In [134]:
# df['sample_duration_code'].unique()

In [135]:
# df[['parameter', 'sample_duration_code']].value_counts()

In [136]:
# df.drop(['Unnamed: 0', 'state_code', 'county_code'], axis=1, inplace=True)

In [137]:
# df.shape

In [138]:
# df['county'].unique()

In [139]:
# df[(df['state'] == 'Alabama') & (df['county'] == 'Shelby')]['pollutant_standard'].unique()

Since the API call failed me, I found that the annual concentration data looked fairly similar to what we were collecting, so giving that a shot.  I downloaded the annual files for years 2006-2021, so we'll take a shot at seeing if this data can be worked into a solution

In [140]:
# # code from Winston merged with approach from
# # https://towardsdatascience.com/pandas-concat-tricks-you-should-know-to-speed-up-your-data-analysis-cd3d4fdfe6dd

# data_path = Path('data/')

# # create a list of all the AQI csv files
# dfs = (
#     pd.read_csv(file, low_memory=False) for file in data_path.glob('*_conc_*') if file.is_file()
# )

# # concatenate the dataframes
# df = pd.concat(dfs)

# # export the final csv
# # res.to_csv('data/aqi_by_year_2006-2021.csv', index=False)

In [141]:
# # in the current data, I think this is akin to completeness_indicator
# df = df[df['completeness_indicator']=='Y']

In [142]:
# df['Year'].value_counts()

In [143]:
# df.columns

In [144]:
# df.columns = [col.lower().replace(' ', '_') for col in df.columns]

In [145]:
# df.columns

In [146]:
# df.isna().sum()[df.isna().sum() > 0]

In [147]:
# df['completeness_indicator'].value_counts()

In [148]:
# df.head()

In [149]:
# df.shape

In [150]:
# births = pd.read_csv('data/cdc_cleaned.csv')

In [151]:
# births.head()

In [152]:
# births.shape

In [153]:
# births.describe()

In [154]:
# births['year'].value_counts()

In [155]:
# set(births[births['year'] == 2007]['county'])

In [156]:
# how many counties from the aqi are also in the cdc data
# len(set(aqi['county']).intersection(set(cdc['County'])))

In [157]:
# what are the differences
#set(aqi['county']).difference(set(cdc['County']))

### EPA API Information

Look to see if getting more granular data from the API is feasible

Your user ID is your email address: mrhurless@gmail.com 
Your key is: rubymouse94 

In [158]:
# import requests

In [159]:
# email = 'mrhurless@gmail.com'
# key = 'rubymouse94'

# url = f'https://aqs.epa.gov/data/api/list/parametersByClass?email={email}&key={key}&pc=CRITERIA'

# res = requests.get(url)

# res.status_code

In [160]:
# # from https://stackoverflow.com/questions/71603314/ssl-error-unsafe-legacy-renegotiation-disabled

# import urllib3
# import ssl

# class CustomHttpAdapter (requests.adapters.HTTPAdapter):
#     # "Transport adapter" that allows us to use custom ssl_context.

#     def __init__(self, ssl_context=None, **kwargs):
#         self.ssl_context = ssl_context
#         super().__init__(**kwargs)

#     def init_poolmanager(self, connections, maxsize, block=False):
#         self.poolmanager = urllib3.poolmanager.PoolManager(
#             num_pools=connections, maxsize=maxsize,
#             block=block, ssl_context=self.ssl_context)


# def get_legacy_session():
#     ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
#     ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
#     session = requests.session()
#     session.mount('https://', CustomHttpAdapter(ctx))
#     return session

In [161]:
# res = get_legacy_session().get(url)

In [162]:
# def get_aqi():
#     creds = ('mhurless@me.com', '0n3L0v3#')
#     email = 'mrhurless@gmail.com'
#     key = 'rubymouse94'
#     all_posts = []
    
#     url = "https://support.brightsign.biz/api/v2/community/posts"
    
#     res = requests.get(url, auth=creds)
    
#     # get data other than posts data
#     metadata = {
#                 key: value for key, value in res.json().items() 
#                 if key != 'posts'
#             }
    
#     last_page = metadata['page_count']
#     current_page = metadata['page']
    
#     #count = 0 #keep track of posts # don't think this is needed with this
#     # approach

#     # goal is to get posts from all pages, so we'll check if we've reached 
#     # the last page, but will also put in a catch for the rate limit as 
#     # mentioned by the API docs
    
#     while current_page <= last_page:
#         res = requests.get(url, auth=creds)

#         if res.status_code == 200:
#             posts = pd.DataFrame(res.json()['posts'])
#             metadata = {
#                 key: value for key, value in res.json().items() 
#                 if key != 'posts'
#             }
            
#             # update current page
#             current_page = metadata['page']
            
#             # update URL to pull posts from next page
#             url = metadata['next_page']

#             all_posts.append(posts)
            
#             #sleep(5)

#             if metadata['page'] == metadata['page_count']: 
#                 break #break loop if last page is reached

#             #get sequential posts from most recent to least    
#             #params['before'] = posts['created_utc'].min()
#         elif res.status_code == 429:
#             sleep(res.headers['retry-after'])
#         else:
#             print(f'status: {res.status_code}')  
#     print(f'posts retrieved from API: {len(all_posts)}')

#     return pd.concat(all_posts)